# get_frequently_mutated Final 

In [1]:
def get_frequently_mutated(cancer_type, cutoff=.1):  
    """take cancer type to import cptac and find the frequently mutated genes in total tumors compared to the cutoff.
        
        Parameters:
        cancer_type (string): type of  cancer 
        cutoff (float): used as comparison to determine status of gene mutation frequency
        
        Returns:
        freq_mutated_df (pd.DataFrame): DataFrame of frequently mutated genes passing the cutoff
            and Total_Mutated (mutated genes / total tumors), percent Truncated, and percent Missence
        
        There are many types of mutations catagorized into the columns Truncated and Missence. 
        The Truncated column includes: Frame_Shift_Del, Frame_Shift_Ins, Splice_Site, Nonsense_Mutation, Nonstop_Mutation
        The Missence column includes: In_Frame_Del, In_Frame_Ins, Missense_Mutation
        These columns count multiple mutations of one gene in the same sample, so percentages in the last two columns may 
        exceed the Total_Mutated column(which only counts if the gene was mutated once)"""    
    
    # import CPTAC and pandas
    
    import pandas as pd
    import cptac
    colon = False
    if cancer_type == "endometrial" or cancer_type == "Endometrial":
        cptac = cptac.Endometrial()
        
    elif cancer_type == "colon" or cancer_type == "Colon":
        import cptac.colon as cptac
        colon = True
        
    elif cancer_type == "ovarian" or cancer_type == "Ovarian":
        import cptac.ovarian as cptac
    
    else:
        str_cancer_options = '\n' + 'Options: endometrial, ovarian, colon'
        print("Please enter a valid cancer type.", str_cancer_options)
        return 0
    
    # get data frames
    somatic_mutations = cptac.get_mutations()
    proteomics = cptac.get_proteomics()
    sample_status_map = cptac._get_sample_status_map()
    merged_mutations = somatic_mutations.join(sample_status_map, how="left") 
    
    # standardize mutation names 
    if colon == True:
        mutation_equivalents = {'frameshift substitution': 'Frame_Shift_Del' , 'frameshift deletion': 'Frame_Shift_Del', 
            'frameshift insertion': 'Frame_Shift_Ins', 'stopgain': 'Nonsense_Mutation ', 'stoploss':'Nonstop_Mutation',
            'nonsynonymous SNV': 'Missense_Mutation','nonframeshift insertion': 'In_Frame_Ins',
            'nonframeshift deletion': 'In_Frame_Del', 'nonframeshift substitution': 'Missense_Mutation'}
        merged_mutations = merged_mutations.replace(to_replace = mutation_equivalents)
        
    # get list of unique genes
    unique_genes = somatic_mutations['Gene'].unique()
    
    # get total tumors/patients
    sample_status_series = sample_status_map.value_counts()
    total_tumor_patients = sample_status_series[0]
        
    # find frequently mutated genes and their total mutated fraction. Create lists for frequently mutated genes and fraction.
    freq_mut = []
    total_fraction_mutated = []
    for gene in unique_genes:
        gene_mutated = merged_mutations.loc[merged_mutations['Gene'] == gene]
        gene_mutated = gene_mutated.index.unique()
        num_gene_mutated = len(gene_mutated)
        fraction = (num_gene_mutated / total_tumor_patients)
        if fraction > cutoff:
            freq_mut.append(gene)
            total_fraction_mutated.append(fraction)
    
    # find truncated fraction
    truncated = []
    for gene in freq_mut:
        gene_mutated = merged_mutations.loc[merged_mutations['Gene'] == gene]
        truncated_df = gene_mutated.loc[(gene_mutated['Mutation'] != 'In_Frame_Del') & 
            (gene_mutated['Mutation'] != 'In_Frame_Ins') & (gene_mutated['Mutation'] != 'Missense_Mutation')] 
        samples_trunc = truncated_df.index.unique()
        num_trunc_mut = len(samples_trunc)
        fraction_trunc = (num_trunc_mut / total_tumor_patients)
        truncated.append(fraction_trunc)

    # find missence fraction 
    missence = []
    for gene in freq_mut:
        gene_mutated = merged_mutations.loc[merged_mutations['Gene'] == gene]
        missence_mutations = gene_mutated.loc[(gene_mutated['Mutation'] == 'In_Frame_Ins') | (gene_mutated['Mutation'] == 'In_Frame_Del') | (gene_mutated['Mutation'] == 'Missense_Mutation')]
        samples_miss = missence_mutations.index.unique()
        num_miss_mut = len(samples_miss)
        fraction_miss = (num_miss_mut / total_tumor_patients)
        missence.append(fraction_miss)
        
    # create dataframe
    merged_lists = list(zip(freq_mut, total_fraction_mutated, truncated, missence))
    freq_mutated_df = pd.DataFrame(merged_lists, columns =['Gene', 'Fraction_Mutated', 'Truncation', 'Missence'])
    freq_mutated_df.name = 'frequently_mutated'
                   
    return freq_mutated_df

Test Endometrial

In [2]:
endo_freq_mutated_df = get_frequently_mutated("endometrial", .15)

Welcome to cptac, a python package for disseminating cancer
proteogenomics data. To view available datasets, enter
'cptac.list_data()'. Extensive tutorials are available at
https://github.com/PayneLab/cptac/tree/master/doc

******
Version: 0.4.3
******
Checking that data files are up-to-date...
100% [..................................................................................] 649 / 649
Data check complete.
endometrial data version: 2.1

Loading acetylproteomics data...
Loading clinical data...
Loading CNA data...
Loading definitions data...
Loading miRNA data...
Loading phosphoproteomics_gene data...
Loading phosphoproteomics_site data...
Loading proteomics data...
Loading somatic data...
Loading somatic_binary data...
Loading transcriptomics_circular data...
Loading transcriptomics_linear data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utiliz

In [3]:
endo_freq_mutated_df


,Gene,Fraction_Mutated,Truncation,Missence
0,RPL22,0.168421,0.157895,0.010526
1,ARID1A,0.452632,0.400000,0.136842
2,PIK3CA,0.494737,0.010526,0.484211
3,FBXW7,0.189474,0.042105,0.157895
4,RBM27,0.168421,0.126316,0.052632
5,NSD1,0.168421,0.115789,0.094737
6,SYNE1,0.168421,0.063158,0.147368
7,ZFHX4,0.200000,0.094737,0.147368
8,PTEN,0.789474,0.568421,0.463158
9,CTCF,0.284211,0.242105,0.094737


Test Colon

In [ ]:
colon_freq_mutated_df = get_frequently_mutated("colon")

You have loaded the cptac colon dataset. To view available dataframes,
use cptac.colon.list_data(). To view available functions for accessing
and manipulating the dataframes, use cptac.colon.list_api().
colon data version: Most recent release

Loading cptac colon data:
Loading clinical data...
Loading miRNA data...
Loading mutation data...
Loading mutation_binary data...
Loading phosphoproteomics_normal data...
Loading phosphoproteomics_tumor data...
Loading proteomics_normal data...
Loading proteomics_tumor data...
Loading transcriptomics data...


In [ ]:
colon_freq_mutated_df

Test Ovarian

In [4]:
ovarian_freq_mutated_df = get_frequently_mutated("ovarian")

Welcome to the cptac data service package. Available datasets may be
viewed using cptac.list_data(). In order to access a specific data
set, import a cptac subfolder using either 'import cptac.dataset' or
'from cptac import dataset'.
******
Version: 0.4.1
******
You have loaded the cptac ovarian dataset. To view available
dataframes, use cptac.ovarian.list_data(). To view available functions
for accessing and manipulating the dataframes, use
cptac.ovarian.list_api().
ovarian data version: Most recent release

Loading cptac ovarian data:
Loading clinical data...
Loading cnv data...
Loading phosphoproteomics data...
Loading proteomics data...
Loading somatic_38 data...
Loading transcriptomics data...
Loading treatment data...

 ******PLEASE READ******
CPTAC is a community resource project and data are made available
rapidly after generation for community research use. The embargo
allows exploring and utilizing the data, but the data may not be in a
publication until June 1, 2019. Please 

In [5]:
ovarian_freq_mutated_df

,Gene,Fraction_Mutated,Truncation,Missence
0,TTN,0.279279,0.135135,0.198198
1,TP53,0.693694,0.243243,0.450450
2,MUC16,0.144144,0.054054,0.108108
3,MT-CO1,0.117117,0.027027,0.090090
4,MT-ND5,0.144144,0.081081,0.063063
5,FCGBP,0.108108,0.054054,0.063063
6,MUC5B,0.108108,0.027027,0.081081
7,MUC4,0.306306,0.135135,0.234234
8,MUC17,0.126126,0.036036,0.108108
9,NCOR2,0.108108,0.081081,0.027027
